In [6]:
import json
import os
import numpy as np
import pandas as pd
from mlcbakery import bakery_client
import dataclasses
import pathlib

def find_or_copy_model(remote_bakery_client, local_bakery_client, collection_name, model_name):
    # check if model exists in local client
    try_local_model = local_bakery_client.get_model_by_name(collection_name, model_name)
    if try_local_model:
        print(f"Model {model_name} already exists in local client")
        return try_local_model
    model = remote_bakery_client.get_model_by_name(collection_name, model_name)
    local_bakery_client.find_or_create_by_collection_name(model.collection_name)
    # origin 
    local_bakery_client.create_model(model.collection_name, model.name, dataclasses.asdict(model))
    return local_bakery_client.get_model_by_name(model.collection_name, model.name)

remote_bakery_client = bakery_client.Client("https://bakery.jetty.io") 
local_bakery_client = bakery_client.Client("http://localhost:8081", token=os.getenv("ADMIN_AUTH_TOKEN"))


In [8]:

clip_model = find_or_copy_model(remote_bakery_client, local_bakery_client,  collection_name="openai", model_name="clip-vit-base-patch16")

compliance_dataset = local_bakery_client.push_dataset(
    dataset_path="compliance/seedlings-tiny",
    metadata={},
    long_description="Seedling compliance images",
    asset_origin="local_training",
    format="image/png",
    data_path="git+ssh://localhost:8081/compliance/seedlings-tiny/main"
)

# define a new model that uses the compliance dataset

compliance_model = local_bakery_client.push_model(
    model_identifier="compliance/seedlings-tiny-vit-base-patch16",
    model_physical_path="git+ssh://localhost:8081/compliance/seedlings-tiny-vit-base-patch16/main",
    model_metadata={},
    model_attributes={},
)

rel = local_bakery_client.create_entity_relationship(
    target_entity_str=f"trained_model/{compliance_model.collection_name}/{compliance_model.name}",
    source_entity_str=f"dataset/{compliance_dataset.collection_name}/{compliance_dataset.name}",
    activity_name="trained",
)
print(rel)

local_bakery_client.create_entity_relationship(
    target_entity_str=f"trained_model/{compliance_model.collection_name}/{compliance_model.name}",
    source_entity_str=f"trained_model/{clip_model.collection_name}/{clip_model.name}",
    activity_name="generated",
)

2025-06-02 23:00:38,523 - mlcbakery.bakery_client - ERROR - Request failed: 404 Client Error: Not Found for url: http://localhost:8081/api/v1/models/openai/clip-vit-base-patch16
2025-06-02 23:00:38,523 - mlcbakery.bakery_client - INFO - Model 'openai/clip-vit-base-patch16' not found.


2025-06-02 23:00:40,859 - mlcbakery.bakery_client - ERROR - Request failed: 404 Client Error: Not Found for url: http://localhost:8081/api/v1/collections/openai
2025-06-02 23:00:40,859 - mlcbakery.bakery_client - WARNING - Could not list collections, attempting to create: Failed to get collection openai: 404 Client Error: Not Found for url: http://localhost:8081/api/v1/collections/openai
2025-06-02 23:00:40,940 - mlcbakery.bakery_client - ERROR - Request failed: 404 Client Error: Not Found for url: http://localhost:8081/api/v1/collections/compliance
2025-06-02 23:00:40,940 - mlcbakery.bakery_client - WARNING - Could not list collections, attempting to create: Failed to get collection compliance: 404 Client Error: Not Found for url: http://localhost:8081/api/v1/collections/compliance
2025-06-02 23:00:40,952 - mlcbakery.bakery_client - ERROR - Request failed: 404 Client Error: Not Found for url: http://localhost:8081/api/v1/datasets/compliance/seedlings-tiny
2025-06-02 23:00:40,952 - mlc

{'id': 1, 'source_entity_id': 2, 'target_entity_id': 3, 'activity_name': 'trained', 'agent_id': None}


{'id': 2,
 'source_entity_id': 1,
 'target_entity_id': 3,
 'activity_name': 'generated',
 'agent_id': None}

In [10]:
local_bakery_client.get_upstream_entities(entity_type="trained_model", collection_name=compliance_model.collection_name, entity_name=compliance_model.name)

{'id': 3,
 'name': 'seedlings-tiny-vit-base-patch16',
 'collection_name': 'compliance',
 'entity_type': 'trained_model',
 'activity_name': None,
 'upstream_entities': [{'id': 2,
   'name': 'seedlings-tiny',
   'collection_name': 'compliance',
   'entity_type': 'dataset',
   'activity_name': 'trained',
   'upstream_entities': [],
   'downstream_entities': []},
  {'id': 1,
   'name': 'clip-vit-base-patch16',
   'collection_name': 'openai',
   'entity_type': 'trained_model',
   'activity_name': 'generated',
   'upstream_entities': [],
   'downstream_entities': []}],
 'downstream_entities': []}

In [11]:
local_bakery_client.get_upstream_entities(entity_type="dataset", collection_name=compliance_dataset.collection_name, entity_name=compliance_dataset.name)

{'id': 2,
 'name': 'seedlings-tiny',
 'collection_name': 'compliance',
 'entity_type': 'dataset',
 'activity_name': None,
 'upstream_entities': [],
 'downstream_entities': [{'id': 3,
   'name': 'seedlings-tiny-vit-base-patch16',
   'collection_name': 'compliance',
   'entity_type': 'trained_model',
   'activity_name': 'trained',
   'upstream_entities': [{'id': 1,
     'name': 'clip-vit-base-patch16',
     'collection_name': 'openai',
     'entity_type': 'trained_model',
     'activity_name': 'generated',
     'upstream_entities': [],
     'downstream_entities': []}],
   'downstream_entities': []}]}

In [16]:
local_bakery_client.push_model(
    "test_collection/test_model",
    "/Users/jon/Projects/mlcbakery/notebooks/test_model.pkl",
    model_metadata={"accuracy": 0.95},
    asset_origin="local_training",
    long_description="A detailed description of the test model.",
    model_attributes={"input_shape": [None, 224, 224, 3]}
)

2025-06-02 23:02:00,141 - mlcbakery.bakery_client - INFO - Updating model test_model in collection test_collection


BakeryModel(id=4, name='test_model', collection_id=3, model_path='/Users/jon/Projects/mlcbakery/notebooks/test_model.pkl', collection_name='test_collection', metadata_version='1.0.0', model_metadata={'accuracy': 0.95}, asset_origin='local_training', long_description='A detailed description of the test model.', model_attributes={'input_shape': [None, 224, 224, 3]}, created_at='2025-06-03T03:01:49.944926Z', parent_collection_model=None)